# 07

In [1]:
import pandas as pd
import numpy as np

In [3]:
macro = pd.read_csv('../data/macro/macro_monthly.csv').iloc[:,1:-1]
target = pd.read_csv('../data/macro/macro_monthly.csv').iloc[:,-1]
lda = pd.read_csv('../data/processed/df_with_lda.csv').iloc[:,-6:]
sentiments = pd.read_csv('../data/processed/sentiments.csv').iloc[1:,:].reset_index().drop(columns=['index'])

In [4]:
final_df = pd.concat([macro, lda, sentiments, target], axis=1).reindex(macro.index)

In [6]:
final_df.head(10)

,DFEDTARL,DFEDTARU,GDPC1,CPIAUCSL,UNRATE,PAYEMS,RRSFS,GFDEBTN,VIXCLS,prob_topic_1,prob_topic_2,prob_topic_3,prob_topic_4,prob_topic_5,prob_topic_6,sentiments,DFF
0,NaN,NaN,15267.026000,199.3,4.7,135429.0,179293.0,8.371156e+06,12.95,2.532629e-07,0.442418,0.098614,1.488264e-01,0.021148,0.288993,-0.1,4.47
1,NaN,NaN,15278.919000,199.4,4.8,135737.0,177887.0,8.387451e+06,12.34,5.791823e-07,0.013136,0.000042,9.707158e-01,0.002117,0.013989,0.0,4.52
2,NaN,NaN,15290.812000,199.7,4.7,136047.0,178100.0,8.403747e+06,11.39,2.570856e-07,0.388304,0.011189,4.659097e-01,0.011551,0.123046,0.0,5.00
3,NaN,NaN,15302.705000,200.7,4.7,136205.0,178088.0,8.420042e+06,11.59,2.403132e-07,0.492659,0.050694,1.158198e-01,0.125781,0.215045,0.0,4.86
4,NaN,NaN,15310.592667,201.3,4.6,136244.0,177200.0,8.449019e+06,16.44,1.027025e-06,0.005665,0.000001,9.923208e-01,0.000148,0.001864,0.6,5.05
5,NaN,NaN,15318.480333,201.8,4.6,136325.0,177343.0,8.477997e+06,13.08,3.974326e-07,0.073057,0.349738,3.974326e-07,0.087838,0.489367,0.0,5.05
6,NaN,NaN,15326.368000,202.9,4.7,136520.0,176937.0,8.506974e+06,14.95,3.292580e-07,0.602091,0.044666,4.766647e-02,0.049294,0.256283,0.0,5.31
7,NaN,NaN,15369.888000,203.8,4.7,136694.0,176983.0,8.564724e+06,12.31,2.579949e-07,0.076280,0.121305,1.086829e-02,0.048339,0.743207,0.1,5.31
8,NaN,NaN,15413.408000,202.8,4.5,136843.0,177002.0,8.622474e+06,11.98,1.209156e-06,0.057376,0.018356,8.510057e-01,0.011598,0.061663,-0.3,5.34
9,NaN,NaN,15456.928000,201.9,4.4,136852.0,177544.0,8.680224e+06,11.10,4.024654e-07,0.127895,0.173201,1.495148e-05,0.348323,0.350566,-0.1,5.31


## Feature Engineering Macro Data

In [7]:
final_df.columns

Index(['DFEDTARL', 'DFEDTARU', 'GDPC1', 'CPIAUCSL', 'UNRATE', 'PAYEMS',
       'RRSFS', 'GFDEBTN', 'VIXCLS', 'prob_topic_1', 'prob_topic_2',
       'prob_topic_3', 'prob_topic_4', 'prob_topic_5', 'prob_topic_6',
       'sentiments', 'DFF'],
      dtype='object')

In [8]:
macro_columns = ['GDPC1',
                'CPIAUCSL',
                'UNRATE',
                'PAYEMS',
                'RRSFS',
                'GFDEBTN',
                'VIXCLS']

In [9]:
for col in macro_columns:
    # percentage difference from the previous idx
    diff = (final_df[col].diff() / final_df[col].shift(1)) * 100
    diff_idx = final_df.columns.get_loc(col) + 1
    
    final_df.insert(loc=diff_idx, column=f'{col}_diff', value=diff)
    
    # simple moving averages (SMA 3, 5)
    sma_3 = final_df[col].rolling(window=3).mean()
    sma_5 = final_df[col].rolling(window=5).mean()
    sma_3_idx = diff_idx + 1
    sma_5_idx = sma_3_idx + 1
    
    final_df.insert(loc=sma_3_idx, column=f'{col}_sma_3', value=sma_3)
    final_df.insert(loc=sma_5_idx, column=f'{col}_sma_5', value=sma_5)
    
    # exponential weighted moving average
    ewm = final_df['GDPC1'].ewm(span=40, adjust=False).mean()
    ewm_idx = sma_5_idx + 1
    
    final_df.insert(loc=ewm_idx, column=f'{col}_ewm', value=ewm)
    
final_df = final_df.dropna().reset_index(drop=True)

In [10]:
final_df.head(10)

,DFEDTARL,DFEDTARU,GDPC1,GDPC1_diff,GDPC1_sma_3,GDPC1_sma_5,GDPC1_ewm,CPIAUCSL,CPIAUCSL_diff,CPIAUCSL_sma_3,...,VIXCLS_sma_5,VIXCLS_ewm,prob_topic_1,prob_topic_2,prob_topic_3,prob_topic_4,prob_topic_5,prob_topic_6,sentiments,DFF
0,0.0,0.25,15213.302333,-0.375637,15270.664667,15361.410600,15496.550343,211.398,-0.823352,213.848667,...,43.042,15496.550343,1.042734e-06,6.659580e-02,1.042734e-06,0.932056,1.042734e-06,1.344982e-03,0.0,0.14
1,0.0,0.25,15155.940000,-0.377054,15213.302333,15281.792533,15479.935204,211.933,0.253077,212.161333,...,47.880,15479.935204,1.800641e-07,5.754637e-01,4.369047e-03,0.294135,5.470247e-04,1.254853e-01,0.1,0.23
2,0.0,0.25,15148.665667,-0.047997,15172.636000,15223.319933,15463.775714,212.705,0.364266,212.012000,...,49.272,15463.775714,2.267387e-07,8.976781e-01,9.070950e-03,0.081272,8.447213e-04,1.113444e-02,-0.1,0.22
3,0.0,0.25,15141.391333,-0.048020,15148.665667,15185.992800,15448.049647,212.495,-0.098728,212.377667,...,46.122,15448.049647,1.306076e-06,3.683760e-02,1.306076e-06,0.956843,2.508194e-05,6.291489e-03,0.1,0.16
4,0.0,0.25,15134.117000,-0.048043,15141.391333,15158.683267,15432.735859,212.709,0.100708,212.636333,...,42.366,15432.735859,2.123809e-07,6.179583e-01,9.674113e-02,0.136572,6.222367e-03,1.425059e-01,0.2,0.20
5,0.0,0.25,15152.485333,0.121370,15142.664556,15146.519867,15419.065102,213.022,0.147149,212.742000,...,40.150,15419.065102,6.926605e-07,6.926605e-07,6.926605e-07,0.999997,6.926605e-07,6.926605e-07,0.1,0.19
6,0.0,0.25,15170.853667,0.121223,15152.485333,15149.502600,15406.957227,214.790,0.829961,213.507000,...,36.452,15406.957227,3.027721e-07,7.972925e-01,5.104788e-02,0.102578,6.437460e-03,4.264385e-02,-0.1,0.22
7,0.0,0.25,15189.222000,0.121076,15170.853667,15157.613867,15396.335997,214.726,-0.029797,214.179333,...,32.366,15396.335997,2.676054e-07,4.800797e-01,1.964681e-01,0.013209,7.138788e-02,2.388548e-01,0.0,0.18
8,0.0,0.25,15244.834000,0.366128,15201.636556,15178.302400,15388.945655,215.445,0.334845,214.987000,...,28.740,15388.945655,5.357356e-07,1.963263e-03,5.357356e-07,0.998035,5.357356e-07,5.357356e-07,0.3,0.15
9,0.0,0.25,15300.446000,0.364792,15244.834000,15211.568200,15384.628599,215.861,0.193089,215.344000,...,26.562,15384.628599,2.542776e-07,6.448919e-01,1.403251e-01,0.102460,1.578810e-02,9.653518e-02,-0.1,0.07
